<a href="https://colab.research.google.com/github/DinurakshanRavichandran/Visio-Glance/blob/XAI/Grad_cam_for_custom_one.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import cv2
from tensorflow.keras.preprocessing.image import load_img, img_to_array

# Load the trained model
model = tf.keras.models.load_model('/content/custom_eye_disease_model.h5', compile=False)

# Select a random index from the validation dataset
random_index = random.randint(0, len(val_generator.filepaths) - 1)
sample_image_path = val_generator.filepaths[random_index]

# Load and preprocess the image
img = load_img(sample_image_path, target_size=(IMG_SIZE, IMG_SIZE))
img_array = img_to_array(img) / 255.0  # Normalize
img_array = np.expand_dims(img_array, axis=0)

# Print the selected index
print(f"Selected validation data index: {random_index}")

# Display the image
plt.imshow(img)
plt.title(f"Selected Image (Index: {random_index})")
plt.axis('off')
plt.show()

# Get the last convolutional layer from one of the base models
# Use the last convolutional layer of the VGG16 model
last_conv_layer_name = "block5_conv3"  # Last convolutional layer in VGG16

# Create a Grad-CAM model
grad_model = tf.keras.models.Model(
    inputs=[model.inputs],  # Dual inputs
    outputs=[model.get_layer(last_conv_layer_name).output, model.output]
)

# Compute gradients for the last convolutional layer
with tf.GradientTape() as tape:
    conv_outputs, predictions = grad_model([img_array, img_array])  # Dual inputs
    predicted_class = tf.argmax(predictions[0])  # Get the predicted class
    loss = predictions[:, predicted_class]  # Loss for the predicted class

# Gradients of the predicted class with respect to the output of the last convolutional layer
grads = tape.gradient(loss, conv_outputs)[0]

# Pool the gradients over all the axes leaving out the channel dimension
pooled_grads = tf.reduce_mean(grads, axis=(0, 1))

# Multiply each channel in the feature map array by the gradient importance
conv_outputs = conv_outputs[0]
heatmap = conv_outputs @ pooled_grads[..., tf.newaxis]
heatmap = tf.squeeze(heatmap)

# Normalize the heatmap
heatmap = tf.maximum(heatmap, 0) / tf.reduce_max(heatmap)

# Resize the heatmap to the original image size
heatmap = cv2.resize(heatmap.numpy(), (IMG_SIZE, IMG_SIZE))

# Convert heatmap to RGB
heatmap = np.uint8(255 * heatmap)
heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)

# Superimpose the heatmap on the original image
superimposed_img = cv2.addWeighted(
    cv2.cvtColor(np.uint8(np.array(img) * 255), cv2.COLOR_RGB2BGR), 0.6,  # Convert to BGR
    heatmap, 0.4, 0
)

# Convert back to RGB for display
superimposed_img = cv2.cvtColor(superimposed_img, cv2.COLOR_BGR2RGB)

# Plot the results
plt.figure(figsize=(10, 5))

# Original Image
plt.subplot(1, 2, 1)
plt.imshow(img)
plt.title("Original Image")
plt.axis('off')

# Grad-CAM Heatmap
plt.subplot(1, 2, 2)
plt.imshow(superimposed_img)
plt.title(f"Grad-CAM (Class: {CATEGORIES[predicted_class]})")
plt.axis('off')

plt.show()